# APS

Code to process, work with and plot APS data to create indicators about skills supply in the UK. 

We are interested in the following indicators:

* Percentage of the population with tertiary education
* Percentage of population employed in professional occupations

Raw collected from https://www.nomisweb.co.uk/articles/676.aspx

See [this table](https://docs.google.com/spreadsheets/d/1V2fAQcvuLsoImwo6uLdyIK3x80pBNoX97CxsxkjvRP4/edit?usp=sharing) for more information.

## Preamble

In [ ]:
import requests 

import numpy as np 
import pandas as pd 
import json
import seaborn as sns

import os
cwd = os.getcwd()

import matplotlib.pyplot as plt
%matplotlib inline

## Data Processing & Transformation

Raw data is downloaded via command line using `get_aps_nomis_data` module in the `beis-indicators/data` directory.
Note: `get_nomis_data` uses `nomis`

### Processing for 'Percentage of population employed in professional occupations' data

In [ ]:
# fetching raw data
data_occupations_json = '../../data/raw/aps_occupations-0-25000.json'
with open(data_occupations_json) as f:
    data = json.load(f)

In [ ]:
df_occupations = pd.DataFrame.from_records(data)

In [ ]:
# selecting the rows with the variable in question
df_occupations = df_occupations[(df_occupations['measures_name']=='Variable')].reset_index(drop=True)

# creating pivot table with indicators as fields
df_occupations_pivot = df_occupations.pivot_table(index=df_occupations[['geography_name', 'geography_code', 'date_code']], columns='variable_name', aggfunc='mean')['obs_value']

In [ ]:
#saving pivot table
df_occupations_pivot.to_csv('../../data/processed/aps/11_11_2019_aps_pro_occupations_data.csv')

### Processing for 'Percentage of the population with tertiary education' data

In [ ]:
# fetching raw data
data_edu_json = '../../data/raw/aps_tertiary_education-0-25000.json'
with open(data_edu_json) as f:
    data = json.load(f)

In [ ]:
df_edu = pd.DataFrame.from_records(data)

In [ ]:
# selecting the rows with the variable in question
df_edu = df_edu[(df_edu['measures_name']=='Variable')].reset_index(drop=True)

# creating pivot table with indicators as fields
df_edu_pivot = df_edu.pivot_table(index=df_edu[['geography_name', 'geography_code', 'date_code']], columns='variable_name', aggfunc='mean')['obs_value']

In [ ]:
#saving pivot table
df_edu_pivot.to_csv('../../data/processed/aps/11_11_2019_aps_tertiary_education_data.csv')

## (Processed) Data Collection

This section includes an example of how to work with the dataset if interested in one year (i.e. 2018).

In [ ]:
data_edu = '../../data/processed/aps/11_11_2019_aps_tertiary_education_data.csv'
data_occupations = '../../data/processed/aps/11_11_2019_aps_pro_occupations_data.csv'

In [ ]:
df_edu = pd.read_csv(data_edu)
df_edu.set_index('geography_name', inplace=True)
df_edu

In [ ]:
df_edu_2018 = df_edu[df_edu['date_code'] == '2018-12']
df_edu_2018

In [ ]:
df_edu_2018['% with NVQ4+ - aged 16-64'].sort_values(ascending=True).plot(kind='barh', figsize=(10,8))
plt.ylabel('NUTS2 Region', fontsize=12)
plt.xlabel('% of NUTS2 Region Population', fontsize=12)
plt.title('Percentage of population in NUTS2 regions with NVQ4+: 2018')

In [ ]:
df_occ = pd.read_csv(data_occupations)
df_occ.set_index('geography_name', inplace=True)
df_occ

In [ ]:
df_occ_2018 = df_occ[df_occ['date_code'] == '2018-12']
# df_occ_2018

In [ ]:
df_occ_2018.plot(kind='barh', figsize=(10,8), stacked=True)
plt.axvline(x= 100, linestyle='--', color='grey', alpha =0.3)
plt.xlabel('% of NUTS2 Region Population')
plt.ylabel('NUTS2 Region')
plt.title('Percentage of population in NUTS2 regions in given employment categories: 2018')
plt.legend(bbox_to_anchor=(1.05, 1.05))